In [38]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras import Sequential
from keras.layers import Dense

In [39]:
def read_table(path):
    """Reads table and drops irrelevant columns and rows.
    Returns an analysis ready DataFrame"""
    df = pd.read_table('EMG/'+path)
    df.drop(columns=['time'],inplace=True)
    df = df[df['class']!=0]
    return df

In [40]:
df = read_table('1/2_1.txt')
for n in range(2,37):
    path = f'{n}/2_{n}.txt'
    df = pd.concat([df, read_table(path)])

In [41]:
df

,channel1,channel2,channel3,channel4,channel5,channel6,channel7,channel8,class
816,-0.00003,-0.00004,-0.00006,-0.00006,-0.00004,-0.00002,-0.00001,-0.00002,1
817,-0.00003,-0.00004,-0.00006,-0.00006,-0.00004,-0.00002,-0.00001,-0.00002,1
818,-0.00003,-0.00004,-0.00006,-0.00006,-0.00004,-0.00002,-0.00001,-0.00002,1
819,-0.00003,-0.00004,-0.00006,-0.00006,-0.00004,-0.00002,-0.00001,-0.00002,1
820,-0.00003,-0.00004,-0.00006,-0.00006,-0.00004,-0.00002,-0.00001,-0.00002,1
...,...,...,...,...,...,...,...,...,...
48664,0.00009,0.00001,0.00003,0.00001,-0.00003,-0.00008,0.00004,0.00000,6
48665,0.00009,0.00001,0.00003,0.00001,-0.00003,-0.00008,0.00004,0.00000,6
48666,0.00009,0.00001,0.00003,0.00001,-0.00003,-0.00008,0.00004,0.00000,6
48667,0.00014,0.00003,-0.00001,0.00001,-0.00012,0.00006,-0.00007,-0.00013,6


In [42]:
df.reset_index(drop=True,inplace=True)

In [56]:
df['class'].value_counts()

class
4    127509
6    127490
5    125881
3    124058
2    121604
1    121539
7      6609
Name: count, dtype: int64

In [57]:
mapping = {1:0, 2:1, 3:2, 4:3, 5:4, 6:5, 7:6}
df['class'] = df['class'].map(mapping)

In [59]:
df['class'].value_counts()

class
3    127509
5    127490
4    125881
2    124058
1    121604
0    121539
6      6609
Name: count, dtype: int64

In [60]:
X = df.drop(columns=['class'])
y = df['class']

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20)

In [62]:
X_train

,channel1,channel2,channel3,channel4,channel5,channel6,channel7,channel8
276642,0.00002,-0.00022,-0.00028,-0.00028,0.00000,-0.00006,-0.00007,0.00004
753913,-0.00006,0.00000,-0.00001,0.00001,0.00004,-0.00020,-0.00008,-0.00019
108991,0.00024,0.00002,-0.00002,-0.00011,-0.00047,-0.00024,-0.00005,0.00011
200723,-0.00005,-0.00008,-0.00010,-0.00013,-0.00028,-0.00015,-0.00004,-0.00001
345527,-0.00011,-0.00020,-0.00003,-0.00001,-0.00003,-0.00003,-0.00005,-0.00022
...,...,...,...,...,...,...,...,...
335732,0.00031,-0.00005,-0.00007,-0.00003,-0.00003,-0.00010,-0.00017,0.00036
373614,-0.00001,-0.00001,-0.00011,0.00001,0.00002,0.00017,-0.00001,0.00001
175647,-0.00012,-0.00009,-0.00002,0.00000,0.00002,0.00007,-0.00015,0.00016
565356,-0.00028,0.00002,0.00004,0.00004,-0.00004,-0.00019,-0.00010,-0.00043


In [63]:
def scaler(df):
    """Normalizes the data, returns a DataFrame with the new scaled numbers"""
    scaler = StandardScaler()
    scaled = scaler.fit_transform(df)
    df_scaled = pd.DataFrame(scaled, columns=df.columns)
    return df_scaled

In [64]:
X_train = scaler(X_train)
X_test = scaler(X_test)

In [67]:
model = Sequential()
model.add(Dense(10, input_dim=8, activation='relu'))
model.add(Dense(10, input_dim=8, activation='relu'))
model.add(Dense(7,activation='softmax'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 10)                90        
                                                                 
 dense_9 (Dense)             (None, 10)                110       
                                                                 
 dense_10 (Dense)            (None, 7)                 77        
                                                                 
Total params: 277
Trainable params: 277
Non-trainable params: 0
_________________________________________________________________


In [69]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, batch_size=30)

Epoch 1/20
20126/20126 [==============================] - 11s 531us/step - loss: 0.9730 - accuracy: 0.6273
Epoch 2/20
20126/20126 [==============================] - 11s 522us/step - loss: 0.9729 - accuracy: 0.6273
Epoch 3/20
20126/20126 [==============================] - 11s 525us/step - loss: 0.9730 - accuracy: 0.6273
Epoch 4/20
20126/20126 [==============================] - 10s 498us/step - loss: 0.9729 - accuracy: 0.6272
Epoch 5/20
11755/20126 [================>.............] - ETA: 4s - loss: 0.9747 - accuracy: 0.6263

KeyboardInterrupt: 

In [70]:
model.evaluate(X_test,y_test)

4717/4717 [==============================] - 2s 423us/step - loss: 0.9733 - accuracy: 0.6278


[0.9733497500419617, 0.6277875900268555]